**TALLER**

**Problema elegido: 8-Puzzle.**

El 8-puzzle es un rompecabezas con un tablero de 3x3, donde hay 8 piezas numeradas del 1 al 8 y un espacio vacío (representado por el 0). El objetivo es reorganizar las piezas para que lleguen a una configuración final específica, generalmente:

1 2 3

4 5 6

7 8 0


El agente debe encontrar el camino desde un estado inicial hasta el objetivo moviendo las piezas adyacentes al espacio vacío en las direcciones posibles (arriba, abajo, izquierda, derecha).

***Algoritmo elegido: A* (A-star).***

***Justificación para la elección de A*:***

Completitud: A* es completo, es decir, siempre encontrará una solución si existe, ya que explora todos los nodos posibles, pero lo hace de manera informada gracias a la heurística.

Optimalidad: A* garantiza que la solución encontrada es la óptima (la más corta) si la heurística utilizada es admisible (no sobreestima el costo real).

Complejidad en memoria: A* tiene una complejidad de memoria de O(b^d), donde b es el factor de ramificación (número de movimientos posibles) y d es la profundidad de la solución. En el peor de los casos, puede ser costoso en memoria, pero es más eficiente que los algoritmos de búsqueda ciega como BFS.

Complejidad temporal: La complejidad temporal de A* es también O(b^d), aunque en la práctica, la heurística ayuda a reducir significativamente la cantidad de nodos explorados, lo que lo hace más eficiente que BFS o DFS.



***Comparación con otros algoritmos:***

BFS: Aunque garantiza encontrar la solución más corta, es más costoso en términos de memoria y tiempo porque no tiene una heurística que guíe la búsqueda.

DFS: No es completo ni óptimo, ya que podría quedarse atascado en ramas profundas sin encontrar la solución.

Greedy Best-First Search: No garantiza una solución óptima, ya que solo utiliza la heurística sin tener en cuenta el costo acumulado.

***Definición del espacio de estados***

Estados (States): Cada configuración del tablero puede representarse como una tupla de 9 elementos, donde los números del 1 al 8 corresponden a las piezas y el 0 al espacio vacío. Un estado es una combinación de posiciones de las piezas.

**Ejemplo de estado**

(1, 2, 3, 4, 5, 6, 7, 8, 0)  # Estado objetivo


**Acciones (Actions)**: Las acciones consisten en mover el espacio vacío (0) hacia una de las posiciones adyacentes: arriba, abajo, izquierda o derecha. Dependiendo de la posición del espacio vacío, algunas acciones no son válidas (por ejemplo, no puedes moverlo hacia la izquierda si ya está en la primera columna).

**Resultado (Modelo de transición)**: Cada acción genera un nuevo estado, que es el resultado de mover el espacio vacío.

*Si el espacio vacío está en la posición (2,1) (última fila, segunda columna),y lo movemos hacia la izquierda, el nuevo estado será:*
(1, 2, 3, 4, 5, 6, 7, 0, 8)

**Costo de acción (Action cost)**: El costo de cada acción es constante (1 movimiento). No hay costos adicionales por mover las piezas, ya que solo nos interesa el número de movimientos para llegar al objetivo.

**Diseño de la heurística**

**Heurística utilizada: Número de piezas fuera de lugar.**

Esta heurística cuenta cuántas piezas del tablero están fuera de lugar, es decir, cuántas no están en su posición correcta.

Es admisible (no sobreestima el costo real) y consistente (satisface la propiedad de consistencia que garantiza que el costo de mover a un vecino nunca es mayor que el costo estimado por la heurística).

Alternativamente, se podría usar la distancia Manhattan, que es la suma de las distancias absolutas entre las posiciones actuales y las posiciones correctas de cada pieza.

# Codigo desarrollado

In [6]:
import heapq

# Estado objetivo
GOAL_STATE = (1, 2, 3, 4, 5, 6, 7, 8, 0)

# Movimiento en las 4 direcciones posibles (arriba, abajo, izquierda, derecha)
MOVES = [(-3, 'up'), (3, 'down'), (-1, 'left'), (1, 'right')]

# Función de heurística: número de piezas fuera de lugar
def misplaced_tiles(state):
    return sum(1 for i in range(9) if state[i] != GOAL_STATE[i] and state[i] != 0)

# Imprimir estado como tablero 3x3
def print_board(state):
    for i in range(0, 9, 3):
        print(state[i:i+3])
    print()

# Clase que representa el problema del 8-puzzle
class Puzzle:
    def __init__(self, initial_state):
        self.initial_state = initial_state
        self.initial_blank_pos = initial_state.index(0)

    @staticmethod
    def is_solvable(state):
        inversion_count = 0
        tiles = [tile for tile in state if tile != 0]
        for i in range(len(tiles)):
            for j in range(i + 1, len(tiles)):
                if tiles[i] > tiles[j]:
                    inversion_count += 1
        return inversion_count % 2 == 0

    def get_neighbors(self, state):
        neighbors = []
        blank_pos = state.index(0)

        for move, direction in MOVES:
            new_blank_pos = blank_pos + move

            if direction == 'up' and blank_pos >= 3:
                pass
            elif direction == 'down' and blank_pos <= 5:
                pass
            elif direction == 'left' and blank_pos % 3 != 0:
                pass
            elif direction == 'right' and blank_pos % 3 != 2:
                pass
            else:
                continue

            new_state = list(state)
            new_state[blank_pos], new_state[new_blank_pos] = new_state[new_blank_pos], new_state[blank_pos]
            neighbors.append((tuple(new_state), direction))
        return neighbors

    def a_star(self):
        start = self.initial_state
        frontier = []
        heapq.heappush(frontier, (misplaced_tiles(start), 0, start, [], []))  # (f(n), g(n), estado, camino, visitados)
        explored = set()

        while frontier:
            f, g, state, path, visited = heapq.heappop(frontier)

            if state == GOAL_STATE:
                return path, visited + [state], state

            if state in explored:
                continue
            explored.add(state)

            for neighbor, direction in self.get_neighbors(state):
                new_path = path + [direction]
                new_visited = visited + [state]
                heapq.heappush(frontier, (g + 1 + misplaced_tiles(neighbor), g + 1, neighbor, new_path, new_visited))

        return None, None, None  # No se encontró solución

# Estado inicial (puedes cambiar este)
#initial_state = (2, 4, 5, 6, 0, 7, 8, 3, 1)
initial_state = (5, 6, 3, 2, 7, 0, 8, 4, 1)

# Inicializar puzzle
puzzle = Puzzle(initial_state)

# Resolver el puzzle
if puzzle.is_solvable(initial_state):
    solution, visited_states, final_state = puzzle.a_star()
else:
    print("Este estado no es resoluble.")
    solution, visited_states, final_state = None, None, None

# Mostrar resultados
if solution:
    print(f"Camino encontrado: {solution}")
    print(f"Número de nodos explorados: {len(visited_states)}")
    print("Estados visitados:")
    for i, state in enumerate(visited_states):
        if i >= len(visited_states) - 3:
            print_board(state)
        else:
            print(state)
    print("Estado final alcanzado:")
    print_board(final_state)
else:
    print("No se encontró solución.")


Camino encontrado: ['up', 'left', 'left', 'down', 'right', 'right', 'up', 'left', 'down', 'down', 'right', 'up', 'left', 'down', 'left', 'up', 'right', 'right', 'up', 'left', 'left', 'down', 'right', 'right', 'down']
Número de nodos explorados: 26
Estados visitados:
(5, 6, 3, 2, 7, 0, 8, 4, 1)
(5, 6, 0, 2, 7, 3, 8, 4, 1)
(5, 0, 6, 2, 7, 3, 8, 4, 1)
(0, 5, 6, 2, 7, 3, 8, 4, 1)
(2, 5, 6, 0, 7, 3, 8, 4, 1)
(2, 5, 6, 7, 0, 3, 8, 4, 1)
(2, 5, 6, 7, 3, 0, 8, 4, 1)
(2, 5, 0, 7, 3, 6, 8, 4, 1)
(2, 0, 5, 7, 3, 6, 8, 4, 1)
(2, 3, 5, 7, 0, 6, 8, 4, 1)
(2, 3, 5, 7, 4, 6, 8, 0, 1)
(2, 3, 5, 7, 4, 6, 8, 1, 0)
(2, 3, 5, 7, 4, 0, 8, 1, 6)
(2, 3, 5, 7, 0, 4, 8, 1, 6)
(2, 3, 5, 7, 1, 4, 8, 0, 6)
(2, 3, 5, 7, 1, 4, 0, 8, 6)
(2, 3, 5, 0, 1, 4, 7, 8, 6)
(2, 3, 5, 1, 0, 4, 7, 8, 6)
(2, 3, 5, 1, 4, 0, 7, 8, 6)
(2, 3, 0, 1, 4, 5, 7, 8, 6)
(2, 0, 3, 1, 4, 5, 7, 8, 6)
(0, 2, 3, 1, 4, 5, 7, 8, 6)
(1, 2, 3, 0, 4, 5, 7, 8, 6)
(1, 2, 3)
(4, 0, 5)
(7, 8, 6)

(1, 2, 3)
(4, 5, 0)
(7, 8, 6)

(1, 2, 3)
(4, 5, 6)
(7, 8, 

**Presentación de resultados**

Al ejecutar el código, el resultado será el camino encontrado desde el estado inicial hasta el objetivo, junto con el número de nodos explorados y todos los estados intermedios visitados.

**Ejemplo de salida:**

Camino encontrado: ['right', 'down', 'left', 'down']
Número de nodos explorados: 10
Estados visitados:
(1, 2, 3, 4, 5, 6, 7, 0, 8)
(1, 2, 3, 4, 5, 6, 7, 8, 0)
...